In [1]:
#!pip install fairlearn
#!pip install raiwidgets

In [16]:
#pip install os
#from google.colab import drive
import os 
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from timeit import default_timer as timer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn import linear_model
import joblib
from sklearn import metrics

In [4]:
drive.mount('/content/drive')

NameError: name 'drive' is not defined

In [17]:
df = pd.read_csv('../../2.modeling/final_training_testing (1).csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head(2)
sorted_df=df.sort_values(by='Timestamp_bid', ascending=True)


In [15]:
print(len(df))
print(df['has impression'].sum())
print(df['has click'].sum())



532897
121904
87


In [6]:
features1=[#'Timestamp_bid'
  #, 'User-Agent_bid'
  #, 'Region ID_bids_join_impressions'
  'City ID'
  , 'Ad Exchange'
  , 'Ad Slot Width'
  , 'Ad Slot Height'
  , 'Ad Slot Visibility'
  , 'Ad Slot Format'
  , 'Ad Slot Floor Price'
  , 'Bidding Price'
  , 'Advertiser ID'  
  ]

target1='has impression'

## Full Data set

In [7]:
#full datset 
n = len(df)

df_train = df[0:int(n*0.8)]
df_test = df[int(n*0.8):]

X1_train=df_train[features1]
X1_test=df_test[features1]

y1_train=df_train[target1]
y1_test=df_test[target1]



print('X_train:', X1_train.shape
      , '\nX_test: ', X1_test.shape 
      , '\ny_train:', y1_train.shape
      , '\ny_test: ', y1_test.shape)


X_train: (426317, 9) 
X_test:  (106580, 9) 
y_train: (426317,) 
y_test:  (106580,)


## Small dataset

In [8]:
#small sample for quick training
X1_train_v1=df_train[features1][:10000]
X1_test_v1=df_test[features1][10000:11000]

y1_train_v1=df_train[target1][:10000]
y1_test_v1=df_test[target1][10000:11000]

print('X_train v1:', X1_train_v1.shape
      , '\nX_test v1: ', X1_test_v1.shape 
      , '\ny_train v1:', y1_train_v1.shape
      , '\ny_test v1: ', y1_test_v1.shape)

X_train v1: (10000, 9) 
X_test v1:  (1000, 9) 
y_train v1: (10000,) 
y_test v1:  (1000,)


# Load models

In [9]:
baseline=joblib.load('/content/drive/MyDrive/Capstone/ipinyou/final/2.modeling/bid-->imp/baseline.pkl')
logistic_model=joblib.load('/content/drive/MyDrive/Capstone/ipinyou/final/2.modeling/bid-->imp/logistic.pkl')

# Fairlearn dashboard

In [31]:
#from fairlearn.widget import FairlearnDashboard
from raiwidgets import FairnessDashboard

FairnessDashboard(y_true = y1_test_v1,
                   y_pred = logistic_model.predict(X1_test_v1),
                   sensitive_features = X1_test_v1['City ID'],
                   #sensitive_feature_names = ["City ID"]
                   )

Output hidden; open in https://colab.research.google.com to view.

In [44]:
from fairlearn.metrics import MetricFrame

y_true=y1_train_v1
y_pred=logistic_model.predict(X1_train_v1)

grouped_metric= MetricFrame(
    metrics.accuracy_score
    , y_true
    , y_pred
    , sensitive_features=X1_train_v1['City ID']
)

Found 357 subgroups. Evaluation may be slow


In [47]:
grouped_metric.by_group.sort_values()

City ID
36     0
352    0
68     0
372    0
295    0
      ..
371    1
317    1
322    1
382    1
39     1
Name: accuracy_score, Length: 357, dtype: object

In [10]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

constraint = DemographicParity()

mitigator = ExponentiatedGradient(logistic_model, constraint)
mitigator.fit(X1_train_v1, y1_train_v1, sensitive_features=X1_train_v1['City ID'])
y_pred_mitigated = mitigator.predict(X)



/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the 

ValueError: ignored